# ItaData2024
modal example

In [1]:
using MLJ, ModalDecisionTrees
using SoleDecisionTreeInterface, Sole, SoleData
using CategoricalArrays
using DataFrames, JLD2, CSV
using Audio911
using Random
using StatsBase, Catch22

┌ Warning: Your operating system has run out of inotify capacity.
│ Check the current value with `cat /proc/sys/fs/inotify/max_user_watches`.
│ Set it to a higher level with, e.g., `echo 65536 | sudo tee -a /proc/sys/fs/inotify/max_user_watches`.
│ This requires having administrative privileges on your machine (or talk to your sysadmin).
│ See https://github.com/timholy/Revise.jl/issues/26 for more information.
└ @ Revise /home/paso/.julia/packages/Revise/uvGMC/src/packagedef.jl:37
┌ Warning: Your operating system has run out of inotify capacity.
│ Check the current value with `cat /proc/sys/fs/inotify/max_user_watches`.
│ Set it to a higher level with, e.g., `echo 65536 | sudo tee -a /proc/sys/fs/inotify/max_user_watches`.
│ This requires having administrative privileges on your machine (or talk to your sysadmin).
│ See https://github.com/timholy/Revise.jl/issues/26 for more information.
└ @ Revise /home/paso/.julia/packages/Revise/uvGMC/src/packagedef.jl:37
┌ Warning: Your operating 

### Open .jld2 file
the file contains 504 samples of respiratory sound, labeled with 2 classes: healty and pneumonia

In [2]:
ds_path = "/datasets/respiratory_Healthy_Pneumonia"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

### Audio features extraction function
This function is called for every audio sample and extract 51 features:
26 bands of the mel spectrogram,
13 coefficients of the mfcc
12 spectral features: centroid, crest, entropy, f0, flatness, flux, kurtosis, rolloff, skewness, decrease, slope, spread

In [3]:
nan_replacer!(x::AbstractArray{Float64}) = replace!(x, NaN => 0.0)

function afe(x::AbstractVector{Float64}; get_only_melfreq=false)
    # -------------------------------- parameters -------------------------------- #
    # audio module
    sr = 8000
    norm = true
    speech_detection = false
    # stft module
    # stft_length = 256
    # win_type = (:hann, :periodic)
    # win_length = 256
    # overlap_length = 128
    stft_length = 1024
    win_type = (:hann, :periodic)
    win_length = 1024
    overlap_length = 512
    stft_norm = :power                      # :power, :magnitude, :pow2mag
    # mel filterbank module
    nbands = 26
    scale = :mel_htk                        # :mel_htk, :mel_slaney, :erb, :bark
    melfb_norm = :bandwidth                 # :bandwidth, :area, :none
    freq_range = (300, round(Int, sr / 2))
    # mel spectrogram module
    db_scale = false
    # mfcc module
    ncoeffs = 13
    rectification = :log                    # :log, :cubic_root
    dither = true
    # f0 module
    method = :nfc
    f0_range = (50, 400)

    # --------------------------------- functions -------------------------------- #
    # audio module
    audio = load_audio(
        file=x,
        sr=sr,
        norm=norm,
    );

    stftspec = get_stft(
        audio=audio,
        stft_length=stft_length,
        win_type=win_type,
        win_length=win_length,
        overlap_length=overlap_length,
        norm=stft_norm
    );

    # mel filterbank module
    melfb = get_melfb(
        stft=stftspec,
        nbands=nbands,
        scale=scale,
        norm=melfb_norm,
        freq_range=freq_range
    );

    if get_only_melfreq
        return melfb.freq
    end

    # mel spectrogram module
    melspec =  get_melspec(
        stft=stftspec,
        fbank=melfb,
        db_scale=db_scale
    );

    # mfcc module
    mfcc = get_mfcc(
        source=melspec,
        ncoeffs=ncoeffs,
        rectification=rectification,
        dither=dither,
    );

    # f0 module
    f0 = get_f0(
        source=stftspec,
        method=method,
        freq_range=f0_range
    );

    # spectral features module
    spect = get_spectrals(
        source=stftspec,
        freq_range=freq_range
    );

    x_features = hcat(
        melspec.spec',
        mfcc.mfcc',
        f0.f0,
        spect.centroid,
        spect.crest,
        spect.entropy,
        spect.flatness,
        spect.flux,
        spect.kurtosis,
        spect.rolloff,
        spect.skewness,
        spect.decrease,
        spect.slope,
        spect.spread
    );

    nan_replacer!(x_features)

    return x_features
end

afe (generic function with 1 method)

### Compute DataFrame of features

In [4]:
color_code = Dict(:red => 31, :green => 32, :yellow => 33, :blue => 34, :magenta => 35, :cyan => 36)
freq = round.(Int, afe(x[1, :audio]; get_only_melfreq=true))
r_select = r"\e\[\d+m(.*?)\e\[0m"
variable_names = [
    ["\e[$(color_code[:yellow])mmel$i=$(freq[i])Hz\e[0m" for i in 1:26]...,
    ["\e[$(color_code[:red])mmfcc$i\e[0m" for i in 1:13]...,
    "\e[$(color_code[:green])mf0\e[0m", "\e[$(color_code[:cyan])mcntrd\e[0m", "\e[$(color_code[:cyan])mcrest\e[0m",
    "\e[$(color_code[:cyan])mentrp\e[0m", "\e[$(color_code[:cyan])mflatn\e[0m", "\e[$(color_code[:cyan])mflux\e[0m",
    "\e[$(color_code[:cyan])mkurts\e[0m", "\e[$(color_code[:cyan])mrllff\e[0m", "\e[$(color_code[:cyan])mskwns\e[0m",
    "\e[$(color_code[:cyan])mdecrs\e[0m", "\e[$(color_code[:cyan])mslope\e[0m", "\e[$(color_code[:cyan])msprd\e[0m"
]

X = DataFrame([name => Vector{Float64}[] for name in [match(r_select, v)[1] for v in variable_names]])

features = [
    maximum,
    minimum,
    StatsBase.mean,
    median,
    # std,
    # Catch22.SB_BinaryStats_mean_longstretch1,
    # Catch22.SB_BinaryStats_diff_longstretch0,
    # Catch22.SB_MotifThree_quantile_hh,
    # Catch22.SB_TransitionMatrix_3ac_sumdiagcov,
]

for i in 1:nrow(x)
    push!(X, collect(eachcol(afe(x[i, :audio]))))
end

yc = CategoricalArray(y);

### Data compression for modal analysis

In [5]:
train_ratio = 0.8
train, test = partition(eachindex(yc), train_ratio, shuffle=true)
# train, test = partition(eachindex(yc), train_ratio, shuffle=false) ### Debug
X_train, y_train = X[train, :], yc[train]
X_test, y_test = X[test, :], yc[test]

println("Training set size: ", size(X_train), " - ", length(y_train))
println("Test set size: ", size(X_test), " - ", length(y_test))

Training set size: (403, 51) - 403
Test set size: (101, 51) - 101


### Train a model

In [6]:
learned_dt_tree = begin
    # model = ModalDecisionTree(; relations = :IA7, features = features)
    model = ModalDecisionTree(; relations = :IA7, features = features)
    mach = machine(model, X_train, y_train) |> fit!
end

report(learned_dt_tree).printmodel(variable_names_map=variable_names);

┌ Info: Precomputing logiset...
└ @ ModalDecisionTrees.MLJInterface /home/paso/.julia/dev/ModalDecisionTrees.jl/src/interfaces/MLJ/wrapdataset.jl:135
┌ Info: Training machine(ModalDecisionTree(max_depth = nothing, …), …).
└ @ MLJBase /home/paso/.julia/packages/MLJBase/7nGJF/src/machines.jl:499


▣ ⟨G⟩(max[mfcc3] < 0.09812318146381033)
├✔ ⟨G⟩((max[mfcc3] < 0.09812318146381033) ∧ ⟨L⟩(max[mfcc2] < 2.307492451039466))
│├✔ Healthy
│└✘ ⟨G⟩((max[mfcc3] < 0.09812318146381033) ∧ ⟨G⟩(max[mfcc9] ≥ 0.5976720024804375))
│ ├✔ Healthy
│ └✘ ⟨G⟩((max[mfcc3] < 0.09812318146381033) ∧ ⟨D̅B̅E̅⟩(max[mfcc7] < -0.6642952487692787))
│  ├✔ Healthy
│  └✘ ⟨G⟩((max[mfcc3] < 0.09812318146381033) ∧ ⟨G⟩(max[mfcc13] ≥ 0.14903729485756076))
│   ├✔ ⟨G⟩((max[mfcc3] < 0.09812318146381033) ∧ ⟨G⟩((max[mfcc13] ≥ 0.14903729485756076) ∧ ⟨AO⟩(max[mfcc2] < 2.307492451039466)))
│   │├✔ Healthy
│   │└✘ ⟨G⟩((max[mfcc3] < 0.09812318146381033) ∧ ⟨G⟩((max[mfcc13] ≥ 0.14903729485756076) ∧ (median[mfcc9] < -0.5866367660703533)))
│   │ ├✔ Healthy
│   │ └✘ ⟨G⟩((max[mfcc3] < 0.09812318146381033) ∧ ⟨G⟩((max[mfcc13] ≥ 0.14903729485756076) ∧ ⟨AO⟩(min[skwns] ≥ 3.78403413752257)))
│   │  ├✔ ⟨G⟩((max[mfcc3] < 0.09812318146381033) ∧ ⟨G⟩((max[mfcc13] ≥ 0.14903729485756076) ∧ ⟨AO⟩((min[skwns] ≥ 3.78403413752257) ∧ ⟨A̅O̅⟩(median[mfcc7] < -0

### Model inspection & rule study

In [7]:
_, mtree = report(mach).sprinkle(X_test, y_test)
sole_dt = ModalDecisionTrees.translate(mtree)

printmodel(sole_dt; show_metrics = true, variable_names_map=variable_names);

Applying tree... 100%|███████████████████████████████████| Time: 0:00:02


▣ ⟨G⟩(max[mfcc3] < 0.09812318146381033)
├✔ ⟨G⟩((max[mfcc3] < 0.09812318146381033) ∧ ⟨L⟩(max[mfcc2] < 2.307492451039466))
│├✔ Healthy : (ninstances = 30, ncovered = 30, confidence = 0.97, lift = 1.0)
│└✘ ⟨G⟩((max[mfcc3] < 0.09812318146381033) ∧ ⟨G⟩(max[mfcc9] ≥ 0.5976720024804375))
│ ├✔ Healthy : (ninstances = 6, ncovered = 6, confidence = 0.67, lift = 1.0)
│ └✘ ⟨G⟩((max[mfcc3] < 0.09812318146381033) ∧ ⟨D̅B̅E̅⟩(max[mfcc7] < -0.6642952487692787))
│  ├✔ Healthy : (ninstances = 7, ncovered = 7, confidence = 1.0, lift = 1.0)
│  └✘ ⟨G⟩((max[mfcc3] < 0.09812318146381033) ∧ ⟨G⟩(max[mfcc13] ≥ 0.14903729485756076))
│   ├✔ ⟨G⟩((max[mfcc3] < 0.09812318146381033) ∧ ⟨G⟩((max[mfcc13] ≥ 0.14903729485756076) ∧ ⟨AO⟩(max[mfcc2] < 2.307492451039466)))
│   │├✔ Healthy : (ninstances = 1, ncovered = 1, confidence = 1.0, lift = 1.0)
│   │└✘ ⟨G⟩((max[mfcc3] < 0.09812318146381033) ∧ ⟨G⟩((max[mfcc13] ≥ 0.14903729485756076) ∧ (median[mfcc9] < -0.5866367660703533)))
│   │ ├✔ Healthy : (ninstances = 1, ncovered = 1

### Extract rules that are at least as good as a random baseline model

In [8]:
interesting_rules = listrules(sole_dt, min_lift = 1.0, min_ninstances = 0);
printmodel.(interesting_rules; show_metrics = true, variable_names_map=variable_names);

▣ ⟨G⟩((max[mfcc3] < 0.09812318146381033) ∧ ⟨L⟩(max[mfcc2] < 2.307492451039466))  ↣  Healthy : (ninstances = 101, ncovered = 30, coverage = 0.3, confidence = 0.97, lift = 1.88, natoms = 2)
▣ ⟨G⟩((max[mfcc3] < 0.09812318146381033) ∧ ⟨G⟩(max[mfcc9] ≥ 0.5976720024804375)) ∧ [G]((max[mfcc3] < 0.09812318146381033) → [L](max[mfcc2] ≥ 2.307492451039466))  ↣  Healthy : (ninstances = 101, ncovered = 6, coverage = 0.06, confidence = 0.67, lift = 1.29, natoms = 4)
▣ ⟨G⟩((max[mfcc3] < 0.09812318146381033) ∧ ⟨D̅B̅E̅⟩(max[mfcc7] < -0.6642952487692787)) ∧ [G]((max[mfcc3] < 0.09812318146381033) → [L](max[mfcc2] ≥ 2.307492451039466)) ∧ [G]((max[mfcc3] < 0.09812318146381033) → [G](max[mfcc9] < 0.5976720024804375))  ↣  Healthy : (ninstances = 101, ncovered = 7, coverage = 0.07, confidence = 1.0, lift = 1.94, natoms = 6)
▣ ⟨G⟩((max[mfcc3] < 0.09812318146381033) ∧ ⟨G⟩((max[mfcc13] ≥ 0.14903729485756076) ∧ ⟨AO⟩(max[mfcc2] < 2.307492451039466))) ∧ [G]((max[mfcc3] < 0.09812318146381033) → [L](max[mfcc2] ≥ 2.30

### Simplify rules while extracting and prettify result

In [9]:
interesting_rules = listrules(sole_dt, min_lift = 1.0, min_ninstances = 0, normalize = true);
printmodel.(interesting_rules; show_metrics = true, syntaxstring_kwargs = (; threshold_digits = 2), variable_names_map=variable_names);

▣ ⟨G⟩((max[mfcc3] < 0.1) ∧ ⟨L⟩(max[mfcc2] < 2.31))  ↣  Healthy : (ninstances = 101, ncovered = 30, coverage = 0.3, confidence = 0.97, lift = 1.88, natoms = 2)
▣ ⟨G⟩((max[mfcc3] < 0.1) ∧ ⟨G⟩(max[mfcc9] ≥ 0.6)) ∧ [G]((max[mfcc3] < 0.1) → [L](max[mfcc2] ≥ 2.31))  ↣  Healthy : (ninstances = 101, ncovered = 6, coverage = 0.06, confidence = 0.67, lift = 1.29, natoms = 4)
▣ ⟨G⟩((max[mfcc3] < 0.1) ∧ ⟨D̅B̅E̅⟩(max[mfcc7] < -0.66)) ∧ [G]((max[mfcc3] < 0.1) → [L](max[mfcc2] ≥ 2.31)) ∧ [G]((max[mfcc3] < 0.1) → [G](max[mfcc9] < 0.6))  ↣  Healthy : (ninstances = 101, ncovered = 7, coverage = 0.07, confidence = 1.0, lift = 1.94, natoms = 6)
▣ ⟨G⟩((max[mfcc3] < 0.1) ∧ ⟨G⟩((max[mfcc13] ≥ 0.15) ∧ ⟨AO⟩(max[mfcc2] < 2.31))) ∧ [G]((max[mfcc3] < 0.1) → [L](max[mfcc2] ≥ 2.31)) ∧ [G]((max[mfcc3] < 0.1) → [G](max[mfcc9] < 0.6)) ∧ [G]((max[mfcc3] < 0.1) → [D̅B̅E̅](max[mfcc7] ≥ -0.66))  ↣  Healthy : (ninstances = 101, ncovered = 1, coverage = 0.01, confidence = 1.0, lift = 1.94, natoms = 9)
▣ ⟨G⟩((max[mfcc3] < 0.

### Directly access rule metrics

In [10]:
readmetrics.(listrules(sole_dt; min_lift=1.0, min_ninstances = 0))

13-element Vector{@NamedTuple{ninstances::Int64, ncovered::Int64, coverage::Float64, confidence::Float64, lift::Float64, natoms::Int64}}:
 (ninstances = 101, ncovered = 30, coverage = 0.297029702970297, confidence = 0.9666666666666667, lift = 1.8775641025641026, natoms = 2)
 (ninstances = 101, ncovered = 6, coverage = 0.0594059405940594, confidence = 0.6666666666666666, lift = 1.2948717948717947, natoms = 4)
 (ninstances = 101, ncovered = 7, coverage = 0.06930693069306931, confidence = 1.0, lift = 1.9423076923076923, natoms = 6)
 (ninstances = 101, ncovered = 1, coverage = 0.009900990099009901, confidence = 1.0, lift = 1.9423076923076923, natoms = 9)
 (ninstances = 101, ncovered = 1, coverage = 0.009900990099009901, confidence = 1.0, lift = 1.9423076923076923, natoms = 12)
 (ninstances = 101, ncovered = 1, coverage = 0.009900990099009901, confidence = 1.0, lift = 1.9423076923076923, natoms = 17)
 (ninstances = 101, ncovered = 1, coverage = 0.009900990099009901, confidence = 1.0, lift =

### Show rules with an additional metric (syntax height of the rule's antecedent)

In [11]:
printmodel.(sort(interesting_rules, by = readmetrics); show_metrics = (; round_digits = nothing, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))), variable_names_map=variable_names);

▣ ⟨G⟩((max[mfcc3] < 0.09812318146381033) ∧ ⟨G⟩((max[mfcc13] ≥ 0.14903729485756076) ∧ ⟨AO⟩(max[mfcc2] < 2.307492451039466))) ∧ [G]((max[mfcc3] < 0.09812318146381033) → [L](max[mfcc2] ≥ 2.307492451039466)) ∧ [G]((max[mfcc3] < 0.09812318146381033) → [G](max[mfcc9] < 0.5976720024804375)) ∧ [G]((max[mfcc3] < 0.09812318146381033) → [D̅B̅E̅](max[mfcc7] ≥ -0.6642952487692787))  ↣  Healthy : (ninstances = 101, ncovered = 1, coverage = 0.009900990099009901, confidence = 1.0, lift = 1.9423076923076923, natoms = 9, height = 8)
▣ ⟨G⟩((max[mfcc3] < 0.09812318146381033) ∧ ⟨G⟩((max[mfcc13] ≥ 0.14903729485756076) ∧ (median[mfcc9] < -0.5866367660703533))) ∧ [G]((max[mfcc3] < 0.09812318146381033) → [L](max[mfcc2] ≥ 2.307492451039466)) ∧ [G]((max[mfcc3] < 0.09812318146381033) → [G](max[mfcc9] < 0.5976720024804375)) ∧ [G]((max[mfcc3] < 0.09812318146381033) → [D̅B̅E̅](max[mfcc7] ≥ -0.6642952487692787)) ∧ [G]((max[mfcc3] < 0.09812318146381033) → [G]((max[mfcc13] ≥ 0.14903729485756076) → [AO](max[mfcc2] ≥ 2.3

### Pretty table of rules and their metrics

In [12]:
metricstable(interesting_rules; variable_names_map=variable_names, metrics_kwargs = (; round_digits = nothing, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))))

┌───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬────────────┬────────────┬──────────┬────────────┬────────────┬─────────┬────────┬─────